# Исследование надёжности заёмщиков


## Описание проекта 

Заказчик — кредитный отдел банка. Необходимо  разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


Импорт необходимых библиотек

In [1]:
import pandas as pd

Чтение данных из файла и сохранение их в переменной data

In [2]:
try:
    data = pd.read_csv('/Users/romanvasyliev/Downloads/data.csv')
except:
    data = pd.read_csv('/datasets/data.csv')

Просмотрим общую информацию о данных 

In [3]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим данные в таблице подробнее:
* Всего в наличии 12 столбцов - с типами данных int64, float64, object
* В двух столбцах видим пропуски в данных 

Подробно разберём какие в data столбцы и какую информацию они содержат:

* children — количество детей в семье
* days_employed — трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — образование клиента
* education_id — идентификатор образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — доход в месяц
* purpose — цель получения кредита

 ## Предобработка данных

### Стиль заголовков

На этом этапе мы переименуем столбцы, исключим пропуски, а также проверим данные на наличие дубликатов.

Проверим корректность именования столбцов в таблице для дальнейшего использования:

In [4]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

В названиях столбцов нет пробелов или же ошибок.

### Заполнение пропусков


Проверим данные на наличие пропусков вызовом  метода sum() для суммирования пропущенных значений.

In [5]:
data.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Видим одинаковое количество пропусков в обоих столбцах - возможно это из-за ошибки в заполнении данных, некорректном заполнении или выгрузке данных. Так как оба столбца отражают количественные данные ( стаж работы в днях и доход в месяц) корректнее будет заполнить эти данные средними значениями или же удалить эти данные в случае если эти потерянные данные составляют небольшой процент от общей массы. 

Давайте узнаем процент отсутствующих данных во всем массиве, для этого найдем отношение пропущенных значений к общему количеству строк

Как видно из анализа - 10 процентов достаточно большое количество данных, и простое удаление этих данных возможно внесет большие искажения в окончательный результат. Поэтому давайте заполним отсутствующие значения средними.

Взглянем на первые строки в таблице и оценим данные 

In [6]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Видим необычный набор данных в столбце days_employed - стаж работы. Стаж работы не может быть отрацательным - соответственно наблюдается ошибка в данных и необходимо сменить отрицательные значения на положительные. Так же наблюдаем, что в строке 4 стоит очень большое число - которое не может характеризовать стаж, к тому же оно положительное в отличии от всех остальных и исключительно у пенсионеров.   Проверим гипотезу что только у пенсионеров в данной таблице стоят положительные числа в таблице.

Узнаем общее количество пенсионеров в таблице

In [7]:
data[data['income_type'] == 'пенсионер']['income_type'].count()

3856

Проверим сколько пенсионеров имеют положительный стаж работы 

In [8]:
data[(data['days_employed'] > 0) & (data['income_type'] == 'пенсионер')]['days_employed'].count()

3443

Мы видим, что есть некоторые различия в количестве и скорее всего это как раз из-за отсутствущих данных. Узнаем сколько строк отсутствует для категории пенсионер

In [9]:
data[data['income_type'] == 'пенсионер']['days_employed'].isna().sum()

413

 Как видим - гипотеза о том, что только пенсионеры имеют положительные данные в столбце days_employed подтвердилась, и скорее всего это не человеческий фактор,а ошибка при обработке или заполнении данных пенсионеров

Заменим все отрицательные значения в столбце  days_employed положительными 

In [10]:
data['days_employed'] = abs(data['days_employed'])

In [11]:
# убедимся что обработка прошла успешно и выведем первые 5 сток
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Для удобства проведения операций над данными - заменим отсутствующие данные нулями в столбцах days_employed, total_income

In [12]:
data['days_employed'] = data['days_employed'].fillna(0)

In [13]:
data['total_income'] = data['total_income'].fillna(0)

### Изменение типов данных

Сменим тип данных в столбцах на Int c помощью метода astype() и округлим данные с помощью оператора round()

In [14]:
data['days_employed'] = round(data['days_employed'].astype(int))

In [15]:
data['total_income'] = round(data['total_income'].astype(int))

Из таблицы выше видно, что стаж работы у пенсионера и сотрудника в тысячи раз отличается и замена отсутствующих значений средней по столбцу была бы не корректной.
Чтобы наиболее корректно заменить отсутствущие данные средними значениями - мозьмем эти среднии для каждой категории людей и используем в качестве замены.

Сгруппируем данные по типу занятости income_type и вычислим средние значения столбца days_employed для каждой категории. И сохраним эти данные в переменной mean_income.

In [16]:
mean_income = data.groupby(by='income_type')['days_employed'].mean().round(0)

In [17]:
mean_income

income_type
безработный        366413.0
в декрете            3296.0
госслужащий          3057.0
компаньон            1900.0
пенсионер          325909.0
предприниматель       260.0
сотрудник            2095.0
студент               578.0
Name: days_employed, dtype: float64

После получения таблицы со средними значениями столбца income_type запустим цикл по всему датасету, в котором все отсутствующие значения, приравненные нулю - заменятся средними значениями для каждой категории типа занятости 

In [18]:
for row in range(len(data)):
    if data.loc[row]['days_employed'] == 0 :
        data.loc[row,'days_employed'] = mean_income[data.loc[row]['income_type']]
    

Ту же самую операцию повторим для столбца датасета total_income. 

Найдем средние значения для столбца total_income, сгруппированные по категориям занятости 

In [19]:
mean_days = data.groupby(by = 'income_type')['total_income'].mean().round(0)

In [20]:
mean_days

income_type
безработный        131339.0
в декрете           53829.0
госслужащий        153679.0
компаньон          182195.0
пенсионер          122440.0
предприниматель    249582.0
сотрудник          145342.0
студент             98201.0
Name: total_income, dtype: float64

Повторим операцию заполнения отсутствующих данных средними значениями для каждой категории 

In [21]:
for row in range(len(data)):
    if data.loc[row]['total_income'] == 0 :
        data.loc[row,'total_income'] = mean_days[data.loc[row]['income_type']]

После заполнения данных, убедимся, что пропусков в таблице нет и можно продолжать искать некорректные данные и исправлять их

In [22]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Удаление дубликатов

Проверим список уникальных значений столбца education, так как в данном столбце видно, что одни и те же значения записаны разным размером штрифта.




In [23]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Видим, что одни и те же данные указаны в разном регистре и следует это исправить для правильного анализа данных.
Используем метод .str.lower() для приведения данных в нижний регистр и проверим что остались только уникальные значения 



In [24]:
data['education'] = data['education'].str.lower()

In [25]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

### Дубликаты

Так же давайте проверим наличие дубликатов данных в таблице и удалим их, если они встречаются.

In [26]:
# Посмотрим количество дубликатов 

data.duplicated().sum()

71

In [27]:
# Удалим ненужные дубликаты и восстановим правильную нумерацию индексов

data = data.drop_duplicates().reset_index(drop=True)

В данных могут встречаться артефакты, которые не отражают действительность и появились по какой-то ошибке. Проверим их наличие в столбце  children

In [28]:
data['children'].unique() 

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Явно прослеживается ситуация, когда сразу видно наличие ошибочных данных в датасете - так как наличие отрицательного количества детей, а так же их количество в размере 20 - явно указывает на наличие ошибки в данных. Проверим предположение и сгруппируем данные на столбцу children

In [29]:
data.groupby(by = 'children')['debt'].count()

children
-1        47
 0     14091
 1      4808
 2      2052
 3       330
 4        41
 5         9
 20       76
Name: debt, dtype: int64

 Мы нашли отрацательное значения (-1) у 47 строк данных, а так же то, что у 76 человек в поле дети указано колличество 20, что вызывает сомнения в достоверности данных, так что перепроверим и выведим часть данных этого значения.
Значение -1 это однозначно ошибка и необходимо ее исправить для корректного анализа, скорее всего она имела человеческий фактор и число детей -1 является числом детей в количество одного ребенка. С помощью функции  abs() изменим значение (-1) в положительное (1)

In [30]:
data['children'] = abs(data['children'])

А так же посмотрим на первые 5 строк таблицы о людях, у кого есть 20 детей

In [31]:
data[data['children'] == 20].head(5) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880,21,среднее,1,женат / замужем,0,M,компаньон,0,145334,покупка жилья
720,20,855,44,среднее,1,женат / замужем,0,F,компаньон,0,112998,покупка недвижимости
1074,20,3310,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518,получение образования
2510,20,2714,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474,операции с коммерческой недвижимостью
2940,20,2161,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739,на покупку автомобиля


Мы наблюдаем странные данные - согласно которым - человек в 21 год имеет 20 детей. Это явная ошибка в данных. Скорее всего имело место некорректное заполнение данных, аномалия или ошибка - и детей у этих людей скорее всего 2.
Заменим количество детей 20 на 2 ребенка во всем датасете

In [32]:
data['children'] = data ['children'].replace (20,2)

Проверим итоговый результат обработки данных в колонке children

In [33]:
data.groupby(by = 'children')['debt'].count()

children
0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: debt, dtype: int64

Как видим из анализа вывода данных мы убрали некорректные значения и таблица годится для дальнейшего анализа.
Повторим те же операции над другими столбцами family_status и income_type

In [34]:
data.groupby(by = 'family_status')['debt'].count().sort_values(ascending = False)

family_status
женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: debt, dtype: int64

In [35]:
data.groupby(by = 'income_type')['debt'].count().sort_values(ascending = False)

income_type
сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: debt, dtype: int64

Видим, что повторы отсутствуют и можно перейти в анализу данных в столбце purpose

In [36]:
data.groupby(by = 'purpose')['debt'].count()

purpose
автомобили                                478
автомобиль                                494
высшее образование                        452
дополнительное образование                460
жилье                                     646
заняться высшим образованием              496
заняться образованием                     408
на покупку автомобиля                     471
на покупку подержанного автомобиля        478
на покупку своего автомобиля              505
на проведение свадьбы                     768
недвижимость                              633
образование                               447
операции с жильем                         652
операции с коммерческой недвижимостью     650
операции с недвижимостью                  675
операции со своей недвижимостью           627
покупка жилой недвижимости                606
покупка жилья                             646
покупка жилья для сдачи                   651
покупка жилья для семьи                   638
покупка коммерческой недви

Видим, что имеет место наличие дублирования различных позиций из-за стиля описания.
Напишем функцию, с помощью которой сгруппируем цели получения кредита на 4 основные группы:
* операции с недвижимостью
* операции с автомобилем
* проведение свадьбы
* получение образования

In [37]:
def set_purpose_category (row):
    purpose = row['purpose']
    if 'жил' in purpose:
        return 'операции с недвижимостью'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    elif 'авто' in purpose: 
        return 'операции с автомобилем'
    elif 'свад' in purpose: 
        return 'проведение свадьбы'
    elif 'образ' in purpose: 
        return 'получение образования'

In [38]:
data['purpose_category'] = data.apply(set_purpose_category, axis = 1) 

In [39]:
data['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

In [40]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы


# Проверка гипотез 

### Создание таблиц - словарей

Как видим в таблице присутствуют дублирования колонок education - education_id и family_status и family_status_id


Создадим два новых датафрейма, в которых: каждому уникальному значению из education соответствует уникальное значение education_id — в первом; каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором. Удалим из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id. Новые датафреймы будут "словарями" к которым мы сможем обращаться по идентификатору.

In [41]:
# Возьмем уникальные значения колонки education и присвоим их переменной education_uniq
education_uniq = data['education'].unique()
education_uniq

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

 Далее нам необходимо присвоить каждому значению из переменной education_uniq его соответствующее значение из столбца education_id. Для этого создадим цикл, где каждое значение будем сравнивать со значением из датасета и если нашлось совпадение - то возьмем соответствующее значение из столбца education_id и добавим его в переменную education_id. И чтобы дальше зря не анализаровать таблицу - выйдем из цикла, для того чтобы взять следующее значение из education_uniq


In [42]:
education_id = []
for index in education_uniq:
    for _ in range(len(data)):
        if data.loc[_]['education'] == index :
            education_id.append (data.loc[_]['education_id'])
            break

Теперь объединим две переменные в один датасет education_dict - что-то вроде аналога словаря 

In [43]:
education_dict = pd.DataFrame({'education_id':education_id,'education': education_uniq })

In [44]:
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Повторим такую же операцию для столбцов family_status и family_status_id

In [45]:
family_uniq = data['family_status'].unique()     # Добавим в переменную уникальные значения столбца family_status
family_uniq

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [46]:
# Пройдемся циклом и запишем в переменную family_status_id соответствующие значения для столбца family_status

family_status_id = []
for index in family_uniq:
    for _ in range(len(data)):
        if data.loc[_]['family_status'] == index :
            family_status_id.append (data.loc[_]['family_status_id'])
            break

Создадим новый датафрейм  family_dict и выведем на экран

In [47]:
family_dict = pd.DataFrame({'family_status_id':family_status_id,'family_status': family_uniq })
family_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Теперь можно удалить ненужные столбцы, чтобы исключить переполнения датасета одинаковыми по смыслу данными

### Группировка данных 

In [48]:
data = data.drop(columns = ['education','family_status'])  

Разделим платежеспособность клиентов банка в датасете в столбце total_income на подкатегории и добавим в столбец total_income_category по следующему принципу
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.




Напишем функцию  set_category , которая будет возвращать значения категории клиента в зависимости от его дохода  

In [49]:
def set_category (row):
    income=row['total_income']
    if income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    elif income > 1000001:
        return 'A'

In [50]:
#  Добавим в таблицу столбец с категорией клиента в зависимости от его дохода total_income_category

data['total_income_category'] = data.apply(set_category, axis = 1) 

In [51]:
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,получение образования,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы,C


Сгруппируем данные чтобы посмотреть на какие цели берут кредит заемщики.

In [52]:
data.groupby(by = 'purpose_category')['purpose_category'].count().sort_values(ascending = False)

purpose_category
операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

Как видим большую часть кредитов заемщики берут на операции с недвижимостью.

# Анализ полученных данных

Цель данного проекта узнать влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Кредит выдается на 4 основные цели, которые мы сгруппировали - операции с недвижимостью, с автомобилем,
получение образования и проведение свадьбы. Выделим эти категории в отдельные датасеты и в каждой категории узнаем распределение должников по семейному положению и количеству детей

In [53]:
data_estate = data[data['purpose_category'] == 'операции с недвижимостью']     # данные с операциями по недвижимости
data_avto = data[data['purpose_category'] == 'операции с автомобилем']         # данные с операциями по автомобилям
data_edu = data[data['purpose_category'] == 'получение образования']           # данные по образованию
data_wedd = data[data['purpose_category'] == 'проведение свадьбы']             # данные на проведение свадеб

Сгруппируем датасет по НЕДВИЖИМОСТИ на данные по количеству детей и семейному положению, используя функцию pivot_table и аргументы функции подсчета количества значений, суммы и среднего.

In [54]:
estate_debt = pd.pivot_table(data_estate,index = ['children', 'family_status_id'], values = 'debt', aggfunc = ['sum', len, 'mean'])

Переименуем столбцы таблицы для наглядности и объединим со словарем семейных отношений для удобства анализа, а так же отсортируем таблицу по проценту должников

In [55]:
estate_debt.columns= ['Количество должников','Общее количество клиентов','Процент должников']
estate_debt.join(family_dict.set_index('family_status_id')).sort_values(by='Процент должников', ascending = False)

Количество должников  Общее количество клиентов  \
children family_status_id                                                    
4        4                                    1                          1   
3        4                                    1                          2   
4        0                                    2                         15   
2        1                                    8                         62   
         2                                    2                         16   
1        1                                   32                        261   
         4                                   29                        258   
3        3                                    1                         10   
2        4                                    4                         48   
         0                                   72                        895   
         3                                    4                         50   
0        1                                   51                        649   
3        0                                   11                        141   
1        2                                    4                         52   
0        4                                   95                       1287   
1        0                                  122                       1715   
         3                                   13                        185   
0        3                                   29                        431   
         0                                  279                       4237   
         2                                   22                        470   
3        2                                    0                          3   
         1                                    0                         13   
4        1                                    0                          4   
         2                                    0                          1   
5        0                                    0                          5   

                           Процент должников          family_status  
children family_status_id                                            
4        4                          1.000000  Не женат / не замужем  
3        4                          0.500000  Не женат / не замужем  
4        0                          0.133333        женат / замужем  
2        1                          0.129032       гражданский брак  
         2                          0.125000         вдовец / вдова  
1        1                          0.122605       гражданский брак  
         4                          0.112403  Не женат / не замужем  
3        3                          0.100000              в разводе  
2        4                          0.083333  Не женат / не замужем  
         0                          0.080447        женат / замужем  
         3                          0.080000              в разводе  
0        1                          0.078582       гражданский брак  
3        0                          0.078014        женат / замужем  
1        2                          0.076923         вдовец / вдова  
0        4                          0.073815  Не женат / не замужем  
1        0                          0.071137        женат / замужем  
         3                          0.070270              в разводе  
0        3                          0.067285              в разводе  
         0                          0.065848        женат / замужем  
         2                          0.046809         вдовец / вдова  
3        2                          0.000000         вдовец / вдова  
         1                          0.000000       гражданский брак  
4        1                          0.000000       гражданский брак  
         2                          0.000000         вдовец / вдова  
5        0                          0.000000        женат / замужем

Как видно из анализа - если у клиента 4 ребенка, он не женат и берет кредит на недвижимость - то с вероятностью 100 процентов он задержит выплаты. Самое большое количество должников приходится на группу женатых/замужних клиентов, у которых нет детей - но процент должников в этой группе равен 6,5 процентам. В то же время клиенты - вдовцы или находящиеся в гражданском браке - с тремя, четырьмя или пятью детьми вообще не имеют долгов по платежам.

Повторим те же самые операции по выборке и группировке данных по категории операций с АВТОМОБИЛЯМИ, используя pivot_table, так же переименуем столбцы и отсортируем таблицу по проценту должников

In [56]:
auto_debt = pd.pivot_table(data_avto,index = ['children', 'family_status_id'], values = 'debt', aggfunc = ['sum', len, 'mean'])
auto_debt.columns= ['Количество должников','Общее количество клиентов','Процент должников']
auto_debt.join(family_dict.set_index('family_status_id')).sort_values(by='Процент должников', ascending = False)

Количество должников  Общее количество клиентов  \
children family_status_id                                                    
3        1                                    2                          5   
2        4                                    5                         21   
         2                                    1                          6   
1        1                                   17                        108   
2        3                                    2                         14   
1        4                                   13                         94   
4        0                                    1                          8   
0        4                                   64                        518   
2        0                                   38                        337   
         1                                    4                         37   
0        1                                   28                        282   
1        0                                   68                        691   
0        2                                   19                        201   
1        3                                    6                         70   
0        0                                  119                       1649   
         3                                   13                        195   
3        0                                    3                         50   
1        2                                    0                         11   
3        3                                    0                          1   
         4                                    0                          4   
4        1                                    0                          1   
         3                                    0                          1   
5        0                                    0                          1   
         1                                    0                          1   

                           Процент должников          family_status  
children family_status_id                                            
3        1                          0.400000       гражданский брак  
2        4                          0.238095  Не женат / не замужем  
         2                          0.166667         вдовец / вдова  
1        1                          0.157407       гражданский брак  
2        3                          0.142857              в разводе  
1        4                          0.138298  Не женат / не замужем  
4        0                          0.125000        женат / замужем  
0        4                          0.123552  Не женат / не замужем  
2        0                          0.112760        женат / замужем  
         1                          0.108108       гражданский брак  
0        1                          0.099291       гражданский брак  
1        0                          0.098408        женат / замужем  
0        2                          0.094527         вдовец / вдова  
1        3                          0.085714              в разводе  
0        0                          0.072165        женат / замужем  
         3                          0.066667              в разводе  
3        0                          0.060000        женат / замужем  
1        2                          0.000000         вдовец / вдова  
3        3                          0.000000              в разводе  
         4                          0.000000  Не женат / не замужем  
4        1                          0.000000       гражданский брак  
         3                          0.000000              в разводе  
5        0                          0.000000        женат / замужем  
         1                          0.000000       гражданский брак

Как видно из анализа - если у клиента 3 ребенка, он не женат и цели кредита связаня с автомобилем - то с вероятностью 40 процентов он задержит выплаты. Самое большое количество должников приходится на группу женатых/замужних клиентов, у которых нет детей - но процент должников в этой группе равен 7,2 процентам. В то же время клиенты - вдовцы с одним ребенком ни разу не имели просрочки по кредитам по операциям с автомобилями.


Повторим те же самые операции по выборке и группировке данных по категории операций с ОБРАЗОВАНИЕМ, используя pivot_table, так же переименуем столбцы и отсортируем таблицу по проценту должников

In [57]:
edu_debt = pd.pivot_table(data_edu,index = ['children', 'family_status_id'], values = 'debt', aggfunc = ['sum', len, 'mean'])
edu_debt.columns= ['Количество должников','Общее количество клиентов','Процент должников']
edu_debt.join(family_dict.set_index('family_status_id')).sort_values(by='Процент должников', ascending = False)

Количество должников  Общее количество клиентов  \
children family_status_id                                                    
2        1                                    6                         32   
1        1                                   18                         97   
3        1                                    1                          6   
1        2                                    3                         18   
0        1                                   35                        269   
         4                                   51                        457   
2        0                                   38                        350   
         3                                    2                         19   
1        4                                   10                        102   
         0                                   57                        598   
0        3                                   13                        158   
         0                                  118                       1582   
         2                                   12                        176   
2        4                                    1                         15   
3        0                                    3                         58   
1        3                                    2                         61   
2        2                                    0                          2   
3        2                                    0                          3   
         4                                    0                          2   
4        0                                    0                          6   
         4                                    0                          1   
5        0                                    0                          1   

                           Процент должников          family_status  
children family_status_id                                            
2        1                          0.187500       гражданский брак  
1        1                          0.185567       гражданский брак  
3        1                          0.166667       гражданский брак  
1        2                          0.166667         вдовец / вдова  
0        1                          0.130112       гражданский брак  
         4                          0.111597  Не женат / не замужем  
2        0                          0.108571        женат / замужем  
         3                          0.105263              в разводе  
1        4                          0.098039  Не женат / не замужем  
         0                          0.095318        женат / замужем  
0        3                          0.082278              в разводе  
         0                          0.074589        женат / замужем  
         2                          0.068182         вдовец / вдова  
2        4                          0.066667  Не женат / не замужем  
3        0                          0.051724        женат / замужем  
1        3                          0.032787              в разводе  
2        2                          0.000000         вдовец / вдова  
3        2                          0.000000         вдовец / вдова  
         4                          0.000000  Не женат / не замужем  
4        0                          0.000000        женат / замужем  
         4                          0.000000  Не женат / не замужем  
5        0                          0.000000        женат / замужем

Как видно из анализа - самые большие задолженности имеют клиенты, находящиеся в гражданском браке с количеством детей от одного до трех - 18.75% - 16.66%. Самое большое количество должников приходится на группу женатых/замужних клиентов, у которых нет детей - но процент должников в этой группе равен 7,4 процентам. В то же время клиенты - вдовцы с двумя-тремя детьми (и остальные, согласно таблице) ни разу не имели просрочки по кредитам на образование.

Повторим те же самые операции по выборке и группировке данных по категории операций связанных со СВАДЬБОЙ, используя pivot_table, так же переименуем столбцы и отсортируем таблицу по проценту должников

In [58]:
wedd_debt = pd.pivot_table(data_wedd,index = ['children', 'family_status_id'], values = 'debt', aggfunc = ['sum', len, 'mean'])
wedd_debt.columns= ['Количество должников','Общее количество клиентов','Процент должников']
wedd_debt.join(family_dict.set_index('family_status_id')).sort_values(by='Процент должников', ascending = False)

,,Количество должников,Общее количество клиентов,Процент должников,family_status
children,family_status_id,,,,
3,1,5,32,0.156250,гражданский брак
1,1,51,534,0.095506,гражданский брак
0,1,115,1530,0.075163,гражданский брак
2,1,15,224,0.066964,гражданский брак
4,1,0,3,0.000000,гражданский брак
5,1,0,1,0.000000,гражданский брак


Как видно из анализа - на свадьбу клиенты берут кредит только находящиеся в гражданском браке, самый большой процент задержек приходится на категорию клиентов с 3 детьми - 15.62%. В то же время клиенты в гражданском браке с четырьмя-пятью детьми вообще не имеют задолженностей. Клиенты без детей являются самой большой категорией должников - и равняются 7,5 % в категории клиентов, взявщих кредит на свадьбу.

#  Результаты исследования 

Задачей данного проекта является узнать влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

В процессе иследования надежности заемщиков удалось обнаружить следующие данные:
* 8 процентов заемщиков имеют просроченную задолженность по кредитам
* основная категория должников приходится на женатых/замужних клиентов без детей, это 53%
* в категории кредита - недвижимость - женатых должников без детей 6,5%
* в категории кредита - автомобиль - женатых должников без детей 7,2%
* в категории кредита - обучение - женатых должников без детей 7,4%
* в категории кредита - свадьба - клиенты - только находящиеся в гражданском браке - должников без детей 7,5%
